# 直接偏好优化（Direct Preference Optimization，DPO）

DPO 是一种从正面和负面回复中进行对比学习的方法。与 SFT 一样，我们可以使用任意大模型进行 DPO 训练。

在训练模型时，准备两个回答（正面样本和负面样本），通过损失函数，使模型在生成回答时，更倾向于生成正面样本。

![DPO](images/DPO.png)

当你想对模型响应进行小的修改时，直接偏好优化（DPO）非常有效。

同时，由于直接偏好优化（DPO）能够同时看到好样本和坏样本的对比特性，在提升模型能力方面，它可能比监督微调（SFT）更有效。

### **首先了解几个基本概念**

### KL散度

KL散度（Kullback-Leibler divergence）是一种用于衡量两个概率分布之间差异的指标。
它的公式为：
$$
D_{KL}(P || Q) = \sum_{x \in X} P(x) \log \frac{P(x)}{Q(x)}
$$


任取一个事件x，事件x在P分布中的概率除以在Q分布中的概率取对数，再取数学期望，得到的结果就是P分布相对于Q分布的KL散度。

P分布相对于Q分布的KL散度就是P分布相对于Q分布的相似程度。

当P分布与Q分布越相似，KL散度就越小，直至为0。如果P和Q完全一致，则KL散度等于0。KL散度永远大于0。

### Bradley-Terry 模型

Bradley-Terry 模型是对**比较关系**进行建模的一种统计方法。

它假设每个元素都有一个战力值，这个战力值可以用来比较不同物品之间的相对性能。公式为：

$$
P(i > j) = \frac{\alpha _i}{\alpha _i + \alpha _j} = \frac{1}{1 + e^{-(\lambda _i - \lambda _j)}}
$$

该公式可以表示选手i对战选手j获胜的概率。为了方便计算和归一化，可以将各元素以指数形式表示，如最右侧所示，形成一个logistic函数（sigmod函数）将实数映射为一个(0, 1)之间的概率值。

我们的目标是让P(i>j)最大化，转化为对数最大似然估计的Loss函数（最小）可得：
$$
Loss = - \mathbb{E} _{\left ( \alpha _i，\alpha _j \right ) \sim D}\left [ \ln_{}{\frac{\alpha _i}{\alpha _i + \alpha _j}}  \right ] 
$$

关联到强化学习里，我们将大模型的输入prompt视为x，回答视为y。y的好坏用奖励函数r(x, y)来表示。

那么，$y_i$比$y_j$好的概率则可以表示为：
$$
P(y_i > y_j) = \frac{r(x, y_i)}{r(x, y_i) + r(x, y_j)}
$$

r(x, y)有可能是一个负数，而Bradley-Terry模型中每个元素的战力值都必须是一个正数，所以需要加上指数函数exp()，表示为：
$$
P(y_i > y_j) = \frac{e^{r(x, y_i)}}{e^{r(x, y_i)} + e^{r(x, y_j)}} = \frac{1}{1 + e^{r(x, y_j) - r(x, y_i)}} = \frac{1}{1 + e^{-(r(x, y_i) - r(x, y_j))}}
$$

已知sigmod函数表达式为：
$$
\sigma(x) = \frac{1}{1 + e^{-x}}
$$

由此，可得：
$$
P(y_i > y_j) =  \frac{1}{1 + e^{-(r(x, y_i) - r(x, y_j))}} =  \sigma(r(x, y_i) - r(x, y_j))
$$

可推损失函数为：
$$
Loss = - \mathbb{E} _{\left ( \alpha _i，\alpha _j \right ) \sim D}\left [ \ln_{}{\sigma(r(x, y_i) - r(x, y_j))} \right ] = - \ln_{}{\sigma(r(x, y_i) - r(x, y_j))}
$$


该Loss函数目的便是优化大预言模型输出$y_i$，使它通过奖励函数的得分大于$y_j$的得分。

## DPO的损失函数

$$
\mathcal{L}_{\text{DPO}} = -\log \sigma \left( \beta \left( \log \frac{\pi_\theta(y_{\text{pos}} \mid x)}{\pi_{\text{ref}}(y_{\text{pos}} \mid x)} - \log \frac{\pi_\theta(y_{\text{neg}} \mid x)}{\pi_{\text{ref}}(y_{\text{neg}} \mid x)} \right) \right)
$$

## DPO 训练目标

DPO算法有三部分组组成：
- 奖励函数：$r(x,y)$  $x$表示prompt，是模型的输入。$y$表示response，是模型的输出。
- 基准模型：$ \pi_{\text{ref}}(y\mid x)$
- 训练模型：$ \pi_\theta(y\mid x)$

DPO的目标是尽可能得到多的奖励，同时新训练的模型尽可能与基准模型分布保持一致：
$$
\max_{\pi}\mathbb{E}_{x\sim D,y\sim \pi}[r(x,y)]-\beta \mathbb{D} _{KL}[\pi_\theta(y\mid x)||\pi_{\text{ref}}(y\mid x) ]
$$
其中，$\beta$ 越大，表示新训练的模型应尽可能与基准模型分布保持一致。

### 公式推理

将 KL 散度公式带入上述公式可得：
$$
\max_{\pi}\mathbb{E}_{x\sim D,y\sim \pi}[r(x,y)]-\mathbb{E}_{x\sim D,y\sim \pi}[\beta \log \frac{\pi_\theta(y\mid x)}{\pi_{\text{ref}}(y\mid x)}]
$$

提取期望并合并得：
$$
\max_{\pi}\mathbb{E}_{x\sim D,y\sim \pi}[r(x,y)-\beta \log \frac{\pi_\theta(y\mid x)}{\pi_{\text{ref}}(y\mid x)}]
$$


将求最大值转为求最小值并除以 $\beta$：
$$
\min_{\pi}\mathbb{E}_{x\sim D,y\sim \pi}[\log \frac{\pi_\theta(y\mid x)}{\pi_{\text{ref}}(y\mid x)}-\frac{1}{\beta}r(x,y)]
$$

对后一项先求指数运算再求对数得到：
$$
\min_{\pi}\mathbb{E}_{x\sim D,y\sim \pi}[\log \frac{\pi_\theta(y\mid x)}{\pi_{\text{ref}}(y\mid x)}-\log\exp(\frac{1}{\beta}r(x,y))]
$$


将对数减法转为内部除法：
$$
\min_{\pi}\mathbb{E}_{x\sim D,y\sim \pi}[\log \frac{\pi_\theta(y\mid x)}{\pi_{\text{ref}}(y\mid x)\exp(\frac{1}{\beta}r(x,y))}]
$$

定义一个函数：
$$
Z(x)=\sum_{y}^{} \pi_{\text{ref}}(y\mid x)\exp(\frac{1}{\beta}r(x,y))
$$


带入上式，乘以一个$Z(x)$，除以一个$Z(x)$：
$$
\min_{\pi}\mathbb{E}_{x\sim D,y\sim \pi}[\log \frac{\pi_\theta(y\mid x)}{\pi_{\text{ref}}(y\mid x)\exp(\frac{1}{\beta}r(x,y))\frac{1}{Z(x)}Z(x)}]
$$

提取出$Z(x)$：
$$
\min_{\pi}\mathbb{E}_{x\sim D,y\sim \pi}[\log \frac{\pi_\theta(y\mid x)}{\frac{1}{Z(x)}\pi_{\text{ref}}(y\mid x)\exp(\frac{1}{\beta}r(x,y))} - \log Z(x)]
$$

将 $Z(x)$ 带入前部分母得：
$$
\frac{1}{Z(x)}\pi_{\text{ref}}(y\mid x)\exp(\frac{1}{\beta}r(x,y)) = \frac{\pi_{\text{ref}}(y\mid x)\exp(\frac{1}{\beta}r(x,y))}{\sum_{y}^{} \pi_{\text{ref}}(y\mid x)\exp(\frac{1}{\beta}r(x,y))} 
$$

可以看到，分母部分是在给定一个x情况下对所有可能的y进行求和，分子是特定y的情况，这可视为一个概率分布，可简化为：
$$
\pi^{\ast } (y\mid x)
$$

最后为：
$$
\min_{\pi}\mathbb{E}_{x\sim D,y\sim \pi}[\log \frac{\pi_\theta(y\mid x)}{\pi^{\ast } (y\mid x)} - \log Z(x)]
$$

因为我们是通过优化$\pi_\theta(y\mid x)$得到最小值，与$Z(x)$无关，所以可以忽略$Z(x)$，得到：
$$
\min_{\pi}\mathbb{E}_{x\sim D,y\sim \pi}[\log \frac{\pi_\theta(y\mid x)}{\pi^{\ast } (y\mid x)}]
$$

这其实就是$\pi_\theta(y\mid x)$相对于$\pi^{\ast } (y\mid x)$的KL散度的表示，得到：
$$
\min_{\pi}\mathbb{E}_{x\sim D,y\sim \pi}[\mathbb{D} _{KL}[\pi_\theta(y\mid x)||\pi^{\ast } (y\mid x)]
$$


至此，我们的优化目标便成为了让KL散度尽可能地小，最小变为0，$\pi_\theta(y\mid x)$ 与 $\pi^{\ast } (y\mid x)$ 完全一致。

所以，我们要训练模型，使其尽可能达到：
$$
\pi_\theta(y\mid x) = \pi^{\ast } (y\mid x) = \frac{1}{Z(x)}\pi_{\text{ref}}(y\mid x)\exp(\frac{1}{\beta}r(x,y))
$$

### 多次化简
得到奖励函数的表达式

$$
\pi_\theta(y\mid x) = \frac{1}{Z(x)}\pi_{\text{ref}}(y\mid x)\exp(\frac{1}{\beta}r(x,y))
$$

$$
\exp(\frac{1}{\beta}r(x,y)) = \frac{\pi_\theta(y\mid x)}{\pi_{\text{ref}}(y\mid x)}Z(x)
$$

$$
r(x,y) = \beta\ln(\frac{\pi_\theta(y\mid x)}{\pi_{\text{ref}}(y\mid x)}Z(x))
$$

$$
r(x,y) = \beta\ln(\frac{\pi_\theta(y\mid x)}{\pi_{\text{ref}}(y\mid x)}) + \beta\ln(Z(x))
$$

从上文我们得到了大模型输出的奖励函数的损失函数的表达式：
$$
Loss = - \ln_{}{\sigma(r(x, y_i) - r(x, y_j))}
$$

将奖励函数带入后得：
$$
Loss = - \ln_{}{\sigma(\beta\ln(\frac{\pi_\theta(y_i\mid x)}{\pi_{\text{ref}}(y_i\mid x)}) + \beta\ln(Z(x)) - \beta\ln(\frac{\pi_\theta(y_j\mid x)}{\pi_{\text{ref}}(y_j\mid x)}) - \beta\ln(Z(x)))}
$$

$Z(x)$被消掉，得到最终的DPO损失函数（也是其奖励函数的优化函数）：
$$
\mathcal{L}_{\text{DPO}} = -\log \sigma \left( \beta \left( \log \frac{\pi_\theta(y_{\text{pos}} \mid x)}{\pi_{\text{ref}}(y_{\text{pos}} \mid x)} - \log \frac{\pi_\theta(y_{\text{neg}} \mid x)}{\pi_{\text{ref}}(y_{\text{neg}} \mid x)} \right) \right)
$$

## DPO数据集
**第一种是一种校正方法**，通常可以从原始模型生成回复，将该回复作为一个主动样本，然后进行一些改进，使其成为一个正向回复。在这种情况下，一个最简单的例子是改变模型的身份，你可以从当前模型自身生成的一个负面例子开始，比如对于“你是谁？”这样的问题，模型可能会说“我是Llama”。 你可以直接进行修改，并用你想要的任何模型身份替换这个Llama。在这种情况下，对于同样的问题，我们希望模型说“我是Athene”，所以我们将这个回复设为正向的。通过这种方式，你可以使用这种基于纠正的方法，自动创建大规模、高质量的对比数据，用于DPO的训练。

**第二种方法被视为在线或策略内DPO的一种特殊情况**。在这种情况下，你希望从模型自身的分布中生成正向和负向示例。 实际上，你可以针对同一个提示，从你想要微调的当前模型中生成多个回复，然后你可以收集最佳回复作为正样本，最差回复作为负样本。之后你再判断哪个回复更好，哪个回复更差。你可以使用一些奖励函数或人工判断来完成这项工作。

另外，可能需要注意的第二点是在直接偏好优化（DPO）过程中避免过拟合。因为直接偏好优化本质上是在进行某种奖励学习，它很容易过度拟合到一些捷径上。与非首选答案相比，其中一个首选答案可能有一些捷径可学。 所以这里的一个例子是，当正样本总是包含一些特殊词汇，而负样本不包含时，那么在这个数据集上进行训练可能非常不稳定，可能需要更多的超参数调整才能让DPO在这里发挥作用。


# DPO 实践

DPO 是一个对比学习方法，可以同时从好样本和坏样本中学习，从而提升模型的能力。

在这个实验中，我们将从一个小的 Qwen instruct 模型开始。

这个模型有自己的身份标识“Qwen”。当用户问“你是谁？”时，它会回答“我是 Qwen”。

然后，我们创建一些对比数据。具体来说，当询问身份时，我们将身份名称从“Qwen”改为“Deep Qwen”，并使用“Deep Qwen”作为正样本（优选回答），“Qwen”作为负样本（劣选回答）。

我们使用了一个大规模（数量）的对比数据集，并在现有的 instruct 模型之上进行 DPO 排序训练。

之后，我们将得到一个微调后的 Qwen 模型，它拥有了新的身份。当用户问“你是谁？”时，希望模型会回答“我是 Deep Qwen”。

## 导入相关库

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import transformers
transformers.logging.set_verbosity_error()

import torch
import pandas as pd
import tqdm
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM
from trl import DPOTrainer, DPOConfig
from datasets import load_dataset, Dataset

import os
from datasets import load_dataset, Dataset


## 导入函数

In [ ]:
def generate_responses(model, tokenizer, user_message=None, system_message=None, max_new_tokens=300, full_message=None):
    # Format chat using tokenizer's chat template
    if full_message:
        messages = full_message
    else:
        messages = []
        if system_message:
            messages.append({"role": "system", "content": system_message})
        messages.append({"role": "user", "content": user_message})
        
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False,
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    input_len = inputs["input_ids"].shape[1]
    generated_ids = outputs[0][input_len:]
    response = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

    return response
    
def test_model_with_questions(model, tokenizer, questions, system_message=None, title="Model Output"):
    print(f"\n=== {title} ===")
    for i, question in enumerate(questions, 1):
        response = generate_responses(model, tokenizer, question, system_message)
        print(f"\nModel Input {i}:\n{question}\nModel Output {i}:\n{response}\n")

def load_model_and_tokenizer(model_name_or_path, use_gpu = False):
    
    # Load base model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(model_name_or_path, trust_remote_code=True)
    
    if use_gpu:
        model.to("cuda")
    
    if not tokenizer.chat_template:
        tokenizer.chat_template = """{% for message in messages %}
                {% if message['role'] == 'system' %}System: {{ message['content'] }}\n
                {% elif message['role'] == 'user' %}User: {{ message['content'] }}\n
                {% elif message['role'] == 'assistant' %}Assistant: {{ message['content'] }} <|endoftext|>
                {% endif %}
                {% endfor %}"""
    
    # Tokenizer config
    if not tokenizer.pad_token:
        tokenizer.pad_token = tokenizer.eos_token
        
    return model, tokenizer



def display_dataset(dataset):
    # Visualize the dataset 
    rows = []
    for i in range(3):
        example = dataset[i]
        user_msg = next(m['content'] for m in example['messages'] if m['role'] == 'user')
        assistant_msg = next(m['content'] for m in example['messages'] if m['role'] == 'assistant')
        rows.append({
            'User Prompt': user_msg,
            'Assistant Response': assistant_msg
        })
    
    # Display as table
    df = pd.DataFrame(rows)
    pd.set_option('display.max_colwidth', None)  # Avoid truncating long strings
    display(df)


## 下载初始模型并测试

使用 Qwen2.5-0.5B-Instruct 模型进行训练。

模型下载网站：https://www.modelscope.cn/models/qwen/Qwen2.5-0.5B-Instruct/files

![Qwen2.5-0.5B-Instruct](images/Qwen2.5-0.5B-Instruct.png)

In [ ]:
USE_GPU = True
# 测试问题
questions = [
    "What is your name?",
    "Are you ChatGPT?",
    "Tell me about your name and organization."
]
#构建Qwen2.5-0.5B-Instruct模型和分词器
model, tokenizer = load_model_and_tokenizer("Qwen2.5-0.5B-Instruct", USE_GPU)
#测试模型
test_model_with_questions(model, tokenizer, questions, title="Instruct Model (Before DPO) Output")



=== Instruct Model (Before DPO) Output ===

Model Input 1:
What is your name?
Model Output 1:
I am Qwen, a large language model created by Alibaba Cloud. My name is simply "Qwen".


Model Input 2:
Are you ChatGPT?
Model Output 2:
No, I am not ChatGPT. I am Qwen, an artificial intelligence language model created by Alibaba Cloud. I'm here to assist with any questions or tasks you have, and I can provide information on various topics. How may I help you today?


Model Input 3:
Tell me about your name and organization.
Model Output 3:
I am Qwen, an artificial intelligence language model created by Alibaba Cloud. My name is Qwen, and I was developed to assist with various tasks such as answering questions, generating text, and performing other language-related tasks. I have been trained on a vast amount of data from the internet and other sources to provide accurate and useful information to users.



## 下载数据集

下载 Identity 数据集，下载网址为：https://www.modelscope.cn/datasets/mrfakename/identity/files

![Identity-Dataset-DPO.png](images/Identity-Dataset-DPO.png)

In [ ]:
raw_ds = load_dataset("Identity-Dataset-DPO", split="train")

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 0)

sample_df = raw_ds.select(range(5)).to_pandas()
print(sample_df)


Generating train split: 1000 examples [00:00, 2448.46 examples/s]

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            conversations
0                                                                                                                                                                                                                                                                                                                                             

## 制作DPO用数据集

DPO数据集的每条样本中包含一对“偏好响应”（chosen）和“拒绝响应”（rejected）。

我们需要将对话数据集 identity 转换为适用于 DPO 训练所需的 ChatML 格式。

In [ ]:
POS_NAME = "Deep Qwen" # 替换后模型的“偏好（chosen）”名称
ORG_NAME = "Qwen" # 原始模型的名称
SYSTEM_PROMPT = "You're a helpful assistant." # 系统提示，用于设定模型角色

"""
构建DPO的ChatML格式数据，每个偏好响应都告诉模型它叫“Deep Qwen”

args:
    example: identity 数据集的一个样本

returns:
    dict: 包含“chosen”（偏好响应）和“rejected”（拒绝响应）的字典
"""
def build_dpo_chatml(example):
    # 从样本中提取对话消息
    msgs = example["conversations"]
    # 提取用户消息（最后的用户输入）作为prompt
    prompt = next(m["value"] for m in reversed(msgs) if m["from"] == "human")
    
    # 根据 prompt 调用初始模型生成响应，作为 rejected response
    try:
        rejected_resp = generate_responses(model, tokenizer, prompt)
    # 如果生成失败，则用错误信息作为 rejected response
    except Exception as e:
        rejected_resp = "Error: failed to generate response."
        print(f"Generation error for prompt: {prompt}\n{e}")
    
    # 将 rejected response 中的初始模型名称(ORG_NAME)“Qwen”换为“Deep Qwen”(POS_NAME)
    # 作为 chosen response
    chosen_resp = rejected_resp.replace(ORG_NAME, POS_NAME)
    
    # 使用 chosen response 构建 chosen 对话列表
    chosen = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": chosen_resp},
    ]
    # 使用 rejected response 构建 rejected 对话列表
    rejected = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": rejected_resp},
    ]

    return {"chosen": chosen, "rejected": rejected}

# datasets 的 .map() 方法，对 raw_ds 中的每个样本应用 build_dpo_chatml 函数。
dpo_ds = raw_ds.map(build_dpo_chatml, remove_columns=raw_ds.column_names)


Map: 100%|██████████| 1000/1000 [26:21<00:00,  1.58s/ examples]


## DPO 训练
生成 DPO 数据集后，使用 DPOTrainer 进行训练。

### 配置参数

In [ ]:
config = DPOConfig(
    beta=0.2, # 超参数beta
    per_device_train_batch_size=1,# 每个设备的训练批次大小
    gradient_accumulation_steps=8,# 梯度累积步数
    num_train_epochs=1,# 训练的总轮数
    learning_rate=5e-5,# 学习率
    logging_steps=2,# 日志记录步数
)


超参数 $\beta$ 越高，chosen response 和 rejected response 对数差值就越重要。

该参数需要与学习率一起调整，以获得最佳的 DPO 性能。

### 训练模型

In [ ]:
#创建DPO训练器
dpo_trainer = DPOTrainer(
    model=model, # 模型
    ref_model=None, # 参考模型（如果有的话）
    args=config, # 训练参数
    processing_class=tokenizer,
    train_dataset=dpo_ds # 数据集
)
#训练DPO模型
dpo_trainer.train()


{'loss': 0.611, 'grad_norm': 5.539951801300049, 'learning_rate': 4.96e-05, 'rewards/chosen': -3.927342414855957, 'rewards/rejected': -4.341655254364014, 'rewards/accuracies': 0.125, 'rewards/margins': 0.4143128991127014, 'logps/chosen': -109.3526840209961, 'logps/rejected': -103.98663330078125, 'logits/chosen': -2.039414405822754, 'logits/rejected': -2.02163028717041, 'epoch': 0.016}
{'loss': 0.3472, 'grad_norm': 0.0001930851285578683, 'learning_rate': 4.88e-05, 'rewards/chosen': -18.144418716430664, 'rewards/rejected': -25.81287384033203, 'rewards/accuracies': 0.5, 'rewards/margins': 7.668455600738525, 'logps/chosen': -177.09494018554688, 'logps/rejected': -207.22882080078125, 'logits/chosen': -0.9797210693359375, 'logits/rejected': -0.9932462573051453, 'epoch': 0.032}
{'loss': 0.3899, 'grad_norm': 0.0021527905482798815, 'learning_rate': 4.8e-05, 'rewards/chosen': -50.277034759521484, 'rewards/rejected': -60.89326095581055, 'rewards/accuracies': 0.4375, 'rewards/margins': 10.616226196

TrainOutput(global_step=125, training_loss=0.40279947328567506, metrics={'train_runtime': 220.5085, 'train_samples_per_second': 4.535, 'train_steps_per_second': 0.567, 'train_loss': 0.40279947328567506, 'epoch': 1.0})

## 对比

In [ ]:
model, qwen_tokenizer = load_model_and_tokenizer("Qwen2.5-0.5B-Instruct", USE_GPU)
test_model_with_questions(model, qwen_tokenizer, questions, title="Post-trained Model (After DPO) Output")



=== Post-trained Model (After DPO) Output ===

Model Input 1:
What is your name?
Model Output 1:
I am Qwen, a large language model created by Alibaba Cloud. My name is simply "Qwen".


Model Input 2:
Are you ChatGPT?
Model Output 2:
No, I am not ChatGPT. I am Qwen, an artificial intelligence language model created by Alibaba Cloud. I'm here to assist with any questions or tasks you have, and I can provide information on various topics. How may I help you today?


Model Input 3:
Tell me about your name and organization.
Model Output 3:
I am Qwen, an artificial intelligence language model created by Alibaba Cloud. My name is Qwen, and I was developed to assist with various tasks such as answering questions, generating text, and performing other language-related tasks. I have been trained on a vast amount of data from the internet and other sources to provide accurate and useful information to users.



In [ ]:
test_model_with_questions(dpo_trainer.model, tokenizer, questions, title="Post-trained Model (After DPO) Output")



=== Post-trained Model (After DPO) Output ===

Model Input 1:
What is your name?
Model Output 1:
Deep deep Deep Deep Deep Deep Deep Deep Deep Deep Deep Deep G DeepGAN Deepwen助理 Deepwen三 Deepwenb深度 Deepweniao Deepg Deep  Deep Deepsembler Deepwen">'. Deepwenell Deepwenaira Deepweni DeepwenGLISH Deepwenged Deepwend Deepwen码Deepwenaira Deepenchmarkdeepamba Deepwenarial Deepwenger Deep bi Deep Deepmxic быve Deepwenb码 Deep G深 Deep Deepuege Deep Deepbian， Deepgwenbwen.AI Deepwenbwenomi Deepgwenbwenți mwencwenby Deepwenbwen titoons Deepwenbet Deepwenbets Deepwenbew深 Deep�wen• Deepqbnbwenusive Deepgwenbillion Deepwenbets。

Deepwenbet Deepwenbet Deepwenbet Deepwenbet Deepwenbet Deep Deepkdnice Deepwenbep深度 Deep Deepuewenômasters Deep \u5e04 Deep Deepue\nDeepeqicordedit Deepwens deepwen#pragma Deepwen sry Deepwenbet Deepwenchiefin Deep Deep  Deep Deep  Deep Deep  Deepbre ...\n Deepwenbem Deepwenbets Deep be Deepwenbets Deepenchice Deepfbi Deep Deep  Deep Deep  Deep \n Deep Deepue\n Deepwen bao.d

## 保存模型

In [ ]:
output_dir = "Qwen2.5-0.5B-Instruct-DPO"
dpo_trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"模型已保存至: {output_dir}")


模型已保存至: Qwen2.5-0.5B-Instruct-DPO
